**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

__Answer__:
$\varepsilon$ is a parameter to determine the probability of choosing exploration or exploitation for the current step. The function `act` draws a random float $p \in [0; 1]$ and:
- if $p <= \varepsilon$: exploration is done.
- else ($p > \varepsilon$): eploitation is done.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1  ########################################################## ISN'T IT [:, -2:] ??????

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            raise RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 ########################################################## ISN'T IT [:, -2:] ??????
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20  # set small when debugging
epochs_test=10  # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Answer__:
- The array `position` stores the current position of the agent.
- The array `board` stores the location of cheese (+0.5) and poisonous cells (-1). When the agent moves to a cell, it gets the value of the cell as rewards and the new cell value is set to 0 (empty cell).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """A random action (random integer between 0 and 4 (not included)) is returned)."""
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epochs):
        
        ##### FILL IN HERE
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random' + str(epochs_test-1) + '.mp4'))

Win/lose count 11.0/15.0. Average score (-4.0)
Win/lose count 10.0/17.0. Average score (-5.5)
Win/lose count 10.5/16.0. Average score (-5.5)
Win/lose count 9.5/21.0. Average score (-7.0)
Win/lose count 7.5/7.0. Average score (-5.5)
Win/lose count 10.5/12.0. Average score (-4.833333333333333)
Win/lose count 8.5/16.0. Average score (-5.214285714285714)
Win/lose count 9.5/14.0. Average score (-5.125)
Win/lose count 12.5/21.0. Average score (-5.5)
Win/lose count 6.5/12.0. Average score (-5.5)
Final score: -5.5


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer__:
1 - The action-value function was previously defined as:
\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
\end{equation*}

Which can be rewritten as:
\begin{split}
Q^\pi(s,a) & = E_{p^{\pi}}[r(s, a) + \sum_{t=1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
& = E_{p^{\pi}}[r(s, a) + \gamma\sum_{t=1}^{T}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
& = E_{p^{\pi}}[r(s, a) + \gamma E_{p^{\pi}}[\sum_{t=1}^{T}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']|s_{0}=s,a_{0}=a] \\
& = E_{p^{\pi}}[r(s, a) + Q^\pi(s',a')]|s_{0}=s,a_{0}=a] \\
& = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{split}

<br>

2 - In its explicit form, the action-value function is:
\begin{equation*}
Q^\pi(s,a)=\sum_{s'}P(s'|s,a)[r(s,a,s') + \gamma\sum_{a'}\pi(s',a')Q^\pi(s',a')]
\end{equation*}

An optimal policy can be found by by maximising over $Q^*(s,a)$:
\begin{equation*}
\pi^*(a|s)=\left\{\begin{split}
1 & \; if \; a=\underset{a}{argmax} \; Q^{*}(s, a) \\
0 & \; otherwise
\end{split}\right.
\end{equation*}

Thus for the optimal action-value function, the previous equation becomes:
\begin{equation*}
Q^\pi(s,a)=\sum_{s'}P(s'|s,a)[r(s,a,s') + \gamma \; \underset{a'}{max} \; Q^{*}(s', a')]
\end{equation*}
\begin{split}
Q^\pi(s,a) & = \sum_{s'}P(s'|s,a)[r(s,a,s') + \gamma \; \underset{a'}{max} \; Q^{*}(s', a')] \\
& = E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{split}

<br>

3 - The Bellman optimality equation from the previous question can be rewritten in a recursive way, at each time step $t$ and for a policy $\pi$:
\begin{equation*}
Q^\pi_{t+1}(s,a) = E_{s'\sim \pi(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{\pi}_{t}(s',a')]
\end{equation*}
At each step $t$, let:  
- $s_{t}$ the state
- $r_{t}$ the reward obtained after performing the action $a_{t}$ ($s_{t+1}$ is then observed)
- $y_{t}$ the target
- $Q(s_{t+1},a;\theta)$ the action-value function, depending on parameters $\theta$ to be optimized.

Then the desired target is composed of the reward obtained after performing an action as well as the maximal expected cumulative reward with the current policy, i.e: $y_{t} = r_{t} + \gamma \; \underset{a}{max} \: Q(s_{t+1},a;\theta)$.

Similar to neural network, a gradient descent step is performed between the target and the current action-value function using Mean Squared Error as the loss function, i.e: $(y_{t} - Q(s_{t},a_{t};\theta))^2$.

Finally, instead of sampling the examples from some known data in the case of neural network, they are sampled from the optimal policy, i.e: 
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        """
        Stores the given moves in the memory list. Only keep the newest 'max_memory' elements.
        
        Parameters
        ----------
        m : list
            Whole description of the move, expected: [state, new_state, action, reward, is_game_over]
        """
        if len(self.memory) == self.max_memory:
            del self.memory[0]
        self.memory.append(m)

    def random_access(self):
        """
        Sample a random move from the memory.
        
        Returns
        ----------
        move : list
            Whole description of the move, expected: [state, new_state, action, reward, is_game_over]
        """
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % (epoch-1) == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        """
        Exploitation strategy: selects the best possible action state which maximises the action value
        function computed from the current state.
        
        Returns
        ----------
        action : int
            Best action for the given state.
        """
        current_state_q = self.model.predict(s.reshape(1, 5, 5, self.n_state))
        return np.argmax(current_state_q)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            move = self.memory.random_access()  # Sample a random move stored in the memory
            s, ns, a, r, game_over = move
            input_states[i] = s
            
            # Set Q(s, a) = r if the episode ends at ns, if not set Q(s, a) = r + gamma*maxQ(s', a')
            if game_over:
                ######## FILL IN
                target_q[i, a] = r
            else:
                ######## FILL IN
                next_state_q = self.model.predict(s.reshape(1, 5, 5, self.n_state))
                target_q[i, a] = r + self.discount * np.max(next_state_q)
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ####### FILL IN
        model = Sequential()  # Input should be flattened
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(32, activation='relu', input_dim=5 * 5 * self.n_state))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model  

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train' + str(epochs_train-1) + '.mp4'))

Epoch 000/020 | Loss 0.0003 | Win/lose count 0.5/2.0 (-1.5)
Epoch 001/020 | Loss 0.0101 | Win/lose count 1.0/1.0 (0.0)
Epoch 002/020 | Loss 0.0104 | Win/lose count 4.0/4.0 (0.0)
Epoch 003/020 | Loss 0.0026 | Win/lose count 2.0/4.0 (-2.0)
Epoch 004/020 | Loss 0.0103 | Win/lose count 4.5/5.0 (-0.5)
Epoch 005/020 | Loss 0.0023 | Win/lose count 4.5/6.0 (-1.5)
Epoch 006/020 | Loss 0.0033 | Win/lose count 6.5/19.0 (-12.5)
Epoch 007/020 | Loss 0.0008 | Win/lose count 10.5/13.0 (-2.5)
Epoch 008/020 | Loss 0.0108 | Win/lose count 11.5/10.0 (1.5)
Epoch 009/020 | Loss 0.0135 | Win/lose count 5.5/17.0 (-11.5)
Epoch 010/020 | Loss 0.0242 | Win/lose count 6.5/11.0 (-4.5)
Epoch 011/020 | Loss 0.0138 | Win/lose count 3.5/3.0 (0.5)
Epoch 012/020 | Loss 0.0140 | Win/lose count 7.5/8.0 (-0.5)
Epoch 013/020 | Loss 0.0082 | Win/lose count 3.5/9.0 (-5.5)
Epoch 014/020 | Loss 0.0080 | Win/lose count 2.0/2.0 (0.0)
Epoch 015/020 | Loss 0.0197 | Win/lose count 6.5/5.0 (1.5)
Epoch 016/020 | Loss 0.0042 | Win/los

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', input_shape=(5, 5, self.n_state)))
        model.add(BatchNormalization())
        model.add(Conv2D(8, (3, 3), strides=(1, 1), activation='relu'))
        model.add(BatchNormalization())
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train' + str(epochs_train-1) + '.mp4'))

Epoch 000/020 | Loss 0.1896 | Win/lose count 3.5/4.0 (-0.5)
Epoch 001/020 | Loss 0.0234 | Win/lose count 3.0/3.0 (0.0)
Epoch 002/020 | Loss 0.0056 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/020 | Loss 0.0034 | Win/lose count 4.0/5.0 (-1.0)
Epoch 004/020 | Loss 0.0180 | Win/lose count 3.5/1.0 (2.5)
Epoch 005/020 | Loss 0.0043 | Win/lose count 4.0/4.0 (0.0)
Epoch 006/020 | Loss 0.0018 | Win/lose count 3.5/5.0 (-1.5)
Epoch 007/020 | Loss 0.0032 | Win/lose count 6.0/6.0 (0.0)
Epoch 008/020 | Loss 0.0106 | Win/lose count 1.0/2.0 (-1.0)
Epoch 009/020 | Loss 0.0013 | Win/lose count 1.5/1.0 (0.5)
Epoch 010/020 | Loss 0.0027 | Win/lose count 5.0/2.0 (3.0)
Epoch 011/020 | Loss 0.0058 | Win/lose count 1.0/2.0 (-1.0)
Epoch 012/020 | Loss 0.0119 | Win/lose count 5.5/3.0 (2.5)
Epoch 013/020 | Loss 0.0099 | Win/lose count 2.5/2.0 (0.5)
Epoch 014/020 | Loss 0.0025 | Win/lose count 4.5/5.0 (-0.5)
Epoch 015/020 | Loss 0.0006 | Win/lose count 2.5/4.0 (-1.5)
Epoch 016/020 | Loss 0.0112 | Win/lose count 2.5

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.0/1.0. Average score (2.0)
Win/lose count 1.0/2.0. Average score (0.5)
Win/lose count 0.5/1.0. Average score (0.16666666666666666)
Win/lose count 2.5/3.0. Average score (0.0)
Win/lose count 2.0/1.0. Average score (0.2)
Win/lose count 0.5/0. Average score (0.25)
Win/lose count 1.0/1.0. Average score (0.21428571428571427)
Win/lose count 0/0. Average score (0.1875)
Win/lose count 1.0/2.0. Average score (0.05555555555555555)
Win/lose count 0/0. Average score (0.05)
Final score: 0.05
Test of the FC
Win/lose count 0.5/4.0. Average score (-3.5)
Win/lose count 1.0/2.0. Average score (-2.25)
Win/lose count 1.5/0. Average score (-1.0)
Win/lose count 2.0/2.0. Average score (-0.75)
Win/lose count 1.5/4.0. Average score (-1.1)
Win/lose count 2.0/4.0. Average score (-1.25)
Win/lose count 3.0/2.0. Average score (-0.9285714285714286)
Win/lose count 1.5/4.0. Average score (-1.125)
Win/lose count 0/1.0. Average score (-1.1111111111111112)
Win/lose count 2.0/1.0. Average 

In [16]:
HTML(display_videos('cnn_test' + str(epochs_test-1) + '.mp4'))

In [17]:
HTML(display_videos('fc_test' + str(epochs_test-1) + '.mp4'))

There is not enough exploration: the agent stays too much inside the region he knows. Thus it mainly spends its time on empty cells. Besides, during testing, a major issue is when the agent has taken all the bonus around it, it struggles to go beyond the square it can observe.

The CNN seems to perform better than the FC model.

Increasing the temperature leads to less empty cell (particularly, there is an increase of cells with cheese).

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
            # Decrease epsilon
            agent.set_epsilon(0.999*agent.epsilon)

        # Save as a mp4
        if e % (epoch-1) == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        print(agent.epsilon)
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super().__init__(grid_size, max_time, temperature)
        # Malus positioning grid
        self.malus_position = np.zeros((grid_size + 4, grid_size + 4))
    
    
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            raise RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        # Negative reward for returning to a known position only for training
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.malus_position[self.x, self.y] = 0.1
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore' + str(epochs_train-1) + '.mp4'))

Epoch 000/020 | Loss 0.0166 | Win/lose count 11.0/26.700000000000063 (-15.700000000000063)
0.8178301806491574
Epoch 001/020 | Loss 0.0266 | Win/lose count 10.500000000000004/28.900000000000126 (-18.400000000000123)
0.6688462043806328
Epoch 002/020 | Loss 0.0217 | Win/lose count 9.800000000000004/29.80000000000013 (-20.000000000000124)
0.5470026121551156
Epoch 003/020 | Loss 0.0176 | Win/lose count 12.700000000000006/27.500000000000096 (-14.80000000000009)
0.44735524511437874
Epoch 004/020 | Loss 0.0401 | Win/lose count 2.9/24.000000000000064 (-21.100000000000065)
0.36586062092624033
Epoch 005/020 | Loss 0.0099 | Win/lose count 6.000000000000001/29.60000000000013 (-23.60000000000013)
0.29921185770452
Epoch 006/020 | Loss 0.0304 | Win/lose count 8.000000000000002/28.1000000000001 (-20.1000000000001)
0.24470448763885772
Epoch 007/020 | Loss 0.0223 | Win/lose count 7.700000000000002/32.900000000000155 (-25.200000000000152)
0.20012671533134627
Epoch 008/020 | Loss 0.0215 | Win/lose count 3.

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore' + str(epochs_test-1) + '.mp4'))

Win/lose count 7.5/4.0. Average score (3.5)
Win/lose count 7.5/2.0. Average score (4.5)
Win/lose count 2.5/0. Average score (3.8333333333333335)
Win/lose count 2.5/0. Average score (3.5)
Win/lose count 0/0. Average score (2.8)
Win/lose count 3.0/1.0. Average score (2.6666666666666665)
Win/lose count 4.0/3.0. Average score (2.4285714285714284)
Win/lose count 3.0/0. Average score (2.5)
Win/lose count 2.0/2.0. Average score (2.2222222222222223)
Win/lose count 1.0/0. Average score (2.1)
Final score: 2.1


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***